# Importações


In [1]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from fuzzywuzzy import process

# Connection


connection.py


In [2]:
# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine(
                "mssql+pyodbc:///?odbc_connect=%s" % params
            )
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

db_read.py


In [3]:
# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def create_automacao_query(
        self, table: str, where: str = None, orderby: str = None
    ) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Criando a query e recebendo dados do Banco de Dados


In [4]:
# cSpell: disable=invalid-name
# Dia de hoje para uso no gráfico
now = pd.to_datetime("today")
today = now.strftime("%Y-%m-%d")

# Encontrando primeiro e último dia do mês anterior
first_day_this_month = now.replace(day=1)  # primeiro dia do mês atual
last_day_month = first_day_this_month - timedelta(
    days=1
)  # último dia do mês anterior
first_day_month = last_day_month.replace(day=1)  # primeiro dia do mês anterior

# mantendo o formato de data e removendo o horário
last_day_month = last_day_month.strftime("%Y-%m-%d")
first_day_month = first_day_month.strftime("%Y-%m-%d")
first_day_this_month = first_day_this_month.strftime("%Y-%m-%d")

print(f"Primeiro dia do mês anterior: {first_day_month}")
print(f"Último dia do mês anterior: {last_day_month}")
print(f"Primeiro dia do mês atual: {first_day_this_month}")

# Instancia a classe Read
DB_read = Read()

# Query para obter os dados do mês anterior
query_occ = DB_read.create_automacao_query(
    table="maquina_ocorrencia",
    where=f"data_registro >= '{first_day_this_month}'",
)

query_info = DB_read.create_automacao_query(
    table="maquina_info", where=f"data_registro >= '{first_day_this_month}'"
)

query_maq_cadastro = DB_read.create_automacao_query(
    table="maquina_cadastro",
    orderby="linha, data_registro DESC, hora_registro DESC",
)

# Leitura do Banco de Dados
df_occ = DB_read.get_automacao_data(query_occ)
df_info = DB_read.get_automacao_data(query_info)
df_maq_cadastro = DB_read.get_automacao_data(query_maq_cadastro)

# df_occ.to_html("teste.html")

Primeiro dia do mês anterior: 2023-12-01
Último dia do mês anterior: 2023-12-31
Primeiro dia do mês atual: 2024-01-01


Testes de Saída do Banco de Dados


In [5]:
df_occ

,recno,maquina_id,motivo_id,problema,solucao,data_registro,hora_registro,usuario_id
0,2638,TMF006,12,,,2024-01-02,01:16:52,000453
1,2639,TMF013,08,,,2024-01-02,01:17:04,000453
2,2640,TMF010,08,,,2024-01-02,01:17:20,000453
3,2641,TMF010,12,,,2024-01-02,01:18:45,000453
4,2642,TMF006,12,,,2024-01-03,08:14:58,000807
...,...,...,...,...,...,...,...,...
171,2809,TMF009,03,,,2024-01-08,04:00:04,000688
172,2810,TMF001,03,,,2024-01-08,04:00:12,000688
173,2811,TMF008,03,,,2024-01-08,04:00:20,000688
174,2812,TMF008,12,,,2024-01-08,07:21:53,000453


In [6]:
df_info

,recno,maquina_id,status,ciclo_1_min,ciclo_15_min,contagem_total_ciclos,contagem_total_produzido,turno,data_registro,hora_registro,tempo_parada,tempo_rodando
0,667577,TMF005,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:14.246666,0.0,0.0
1,667578,TMF002,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:15.253333,0.0,0.0
2,667579,TMF015,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:16.253333,0.0,0.0
3,667580,TMF011,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:17.253333,0.0,0.0
4,667581,TMF014,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:18.253333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
73957,741534,TMF006,false,0.0,0.0,0.0,0.0,MAT,2024-01-08,08:05:01.396666,0.0,0.0
73958,741535,TMF013,true,6.0,70.0,44.0,44.0,MAT,2024-01-08,08:05:02.400000,0.0,0.0
73959,741536,TMF012,false,0.0,0.0,0.0,0.0,MAT,2024-01-08,08:05:03.400000,0.0,0.0
73960,741537,TMF007,false,0.0,0.0,0.0,0.0,MAT,2024-01-08,08:05:04.400000,0.0,0.0


In [7]:
df_maq_cadastro

,recno,maquina_id,fabrica,linha,data_registro,hora_registro,usuario_id
0,51,TMF003,,0,2023-11-11,12:41:46,000264
1,48,TMF005,,0,2023-11-11,12:37:43,000264
2,47,TMF014,,0,2023-10-29,11:44:53,000264
3,43,TMF005,,0,2023-10-22,12:32:23,000264
4,42,TMF014,,0,2023-10-14,14:58:15,000264
5,39,TMF005,0,0,2023-10-06,15:53:45,000264
6,38,TMF003,,0,2023-10-02,08:17:14,002131
7,37,TMF003,,0,2023-09-30,16:35:54,000264
8,36,TMF003,,0,2023-09-30,16:35:37,000264
9,35,TMF003,,0,2023-09-30,16:34:26,000264


# Service


clean_data.py


In [47]:
class CleanData:
    """
    Classe para limpeza dos dados

    Atributos:
        df (pd.DataFrame): Dataframe com os dados a serem limpos

    Métodos:
        clean_maq_cadastro: Limpa os dados de cadastro das máquinas

    """

    def __init__(self):
        self.df_cadastro = pd.DataFrame()
        self.df_info = pd.DataFrame()
        self.df_occ = pd.DataFrame()

    def clean_maq_cadastro(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de cadastro das máquinas

        Args:
        -----
            data (pd.DataFrame): Dataframe com os dados a serem limpos

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados limpos

        Exemplo:
        --------

            >>> from app.service.clean_data import CleanData
            >>> import pandas as pd
            >>> df = pd.DataFrame({'maquina_id': [TMF001, TMF002, TMF003], 'linha': [1, 2, 3],
            'fabrica': [1, 2, 2], 'data_registro': ['2021-01-01', '2021-01-01', '2021-01-01'],
            'hora_registro': ['00:00:00.000', '00:00:00.000', '00:00:00.000'], 'recno': [1, 2, 3],
            usuario_id: [00532, 00533, 00534]})
            >>> clean_data = CleanData()
            >>> df_clean = clean_data.clean_maq_cadastro(df)
            >>> df_clean
                maquina_id  linha  fabrica data_hora_registro       usuario_id
            0     TMF001      1        1    2021-01-01 00:00:00      532
            1     TMF002      2        2    2021-01-01 00:00:00      533
            2     TMF003      3        2    2021-01-01 00:00:00      534


        """
        self.df_cadastro = data.copy()

        # Remover rows onde a linha é 0
        self.df_cadastro = self.df_cadastro[self.df_cadastro["linha"] != 0]

        # Remover linhas duplicadas (erros de cadastro)
        self.df_cadastro = self.df_cadastro.drop_duplicates(
            subset=["data_registro", "linha"], keep="first"
        )

        # Criar nova coluna combinando data e hora
        self.df_cadastro["data_hora_registro"] = (
            self.df_cadastro["data_registro"].astype(str)
            + " "
            + self.df_cadastro["hora_registro"]
            .astype(str)
            .str.split(".")
            .str[0]
        )

        # Converter coluna data_hora_registro para datetime
        self.df_cadastro["data_hora_registro"] = pd.to_datetime(
            self.df_cadastro["data_hora_registro"], format="%Y-%m-%d %H:%M:%S"
        )

        # Remover colunas desnecessárias
        self.df_cadastro = self.df_cadastro.drop(
            columns=["recno", "data_registro", "hora_registro"]
        )

        # Ordenar dataframe para facilitar trabalho futuro
        self.df_cadastro = self.df_cadastro.sort_values(
            by=["maquina_id", "data_hora_registro"], ascending=[True, False]
        )

        # reiniciar o index
        self.df_cadastro = self.df_cadastro.reset_index(drop=True)

        return self.df_cadastro

    def clean_maq_info(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de cadastro das máquinas

        Args:
        -----
            data (pd.DataFrame): Dataframe com os dados a serem limpos

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados limpos

        Exemplo:
        --------

            >>> from app.service.clean_data import CleanData
            >>> import pandas as pd
            >>> df = pd.DataFrame({'maquina_id': [TMF001, TMF002, TMF003], 'status'[False, True, True], 'turno'[MAT, VES, NOT], 'data_registro': ['2021-01-01', '2021-01-01', '2021-01-01'],
            'hora_registro': ['00:00:00.000', '00:00:00.000', '00:00:00.000'], 'recno': [1, 2, 3],
            })
            >>> clean_data = CleanData()
            >>> df_clean = clean_data.clean_maq_info(df)
            >>> df_clean
                maquina_id  status  turno   data_hora_registro      data_hora_final         tempo_registro_min
            0     TMF001    parada    MAT   2021-01-01 00:00:00     2021-01-01 00:01:00     480
            1     TMF002    rodando   VES   2021-01-01 00:00:00     2021-01-01 00:01:00     10
            2     TMF003    rodando   NOT   2021-01-01 00:00:00     2021-01-01 00:01:00     10

        """

        self.df_info = data.copy()

        # Ordenar por maquina_id e data_registro, hora_registro
        self.df_info.sort_values(
            by=["maquina_id", "data_registro", "hora_registro"], inplace=True
        )

        # Criar nova coluna combinando data e hora
        self.df_info["data_hora_registro"] = (
            self.df_info["data_registro"].astype(str)
            + " "
            + self.df_info["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Descartar colunas desnecessárias
        self.df_info = self.df_info.drop(
            columns=["recno", "data_registro", "hora_registro"]
        )

        # Corrigindo erros da DB (horário dos turnos) - vai se tornar obsoleto à partir de Fev/24
        self.df_info.sort_values(
            by=["maquina_id", "data_hora_registro"], inplace=True
        )
        self.df_info = self.df_info[
            ~(
                (self.df_info["turno"] == "VES")
                & (
                    self.df_info["maquina_id"]
                    != self.df_info["maquina_id"].shift()
                )
            )
        ]  # descarta a primeira linha se o turno for VES

        # Criar nova coluna status_change para identificar mudança de status
        self.df_info["status_change"] = self.df_info["status"].ne(
            self.df_info["status"].shift()
        )

        # Criar coluna para identificar a mudança de máquina
        self.df_info["maquina_change"] = self.df_info["maquina_id"].ne(
            self.df_info["maquina_id"].shift()
        )

        # criar coluna para consolidar mudança de status e de máquina
        self.df_info["change"] = (
            self.df_info["status_change"] | self.df_info["maquina_change"]
        )

        # Criar coluna para identificar a mudança de turno
        self.df_info["turno_change"] = self.df_info["turno"].ne(
            self.df_info["turno"].shift()
        )

        # Atualizar coluna change para incluir mudança de turno
        self.df_info["change"] = (
            self.df_info["change"] | self.df_info["turno_change"]
        )

        # Agrupar por maquina e identificar data e hora da última mudança de status
        self.df_info["change_time"] = (
            self.df_info.groupby("maquina_id")["data_hora_registro"]
            .shift(0)
            .where(self.df_info["change"])
        )

        # Remover as linhas onde change_time é nulo
        self.df_info.dropna(subset=["change_time"], inplace=True)

        # Criar nova coluna com a data_hora_final do status
        self.df_info["data_hora_final"] = (
            self.df_info.groupby("maquina_id")["data_hora_registro"]
            .shift(-1)
            .where(~self.df_info["maquina_change"])
        )

        # Atualizar coluna data_hora_final onde maquina_change é True
        self.df_info.loc[
            self.df_info["maquina_change"], "data_hora_final"
        ] = self.df_info["change_time"].shift(-1)

        # Remover colunas desnecessárias
        self.df_info.drop(
            columns=[
                "status_change",
                "maquina_change",
                "turno_change",
                "change",
                "change_time",
            ],
            inplace=True,
        )

        # Remover linhas onde data_hora_final é nulo
        self.df_info.dropna(subset=["data_hora_final"], inplace=True)

        # Cria nova coluna tempo_registro_min para calcular o tempo de registro em minutos
        self.df_info["tempo_registro_min"] = (
            pd.to_datetime(self.df_info["data_hora_final"])
            - pd.to_datetime(self.df_info["data_hora_registro"])
        ).dt.total_seconds() / 60

        # Arredondar tempo_registro_min e converter para inteiro
        self.df_info["tempo_registro_min"] = (
            self.df_info["tempo_registro_min"].round(0).astype(int)
        )

        # Incluir um status in_test para os casos onde o status true permanece por menos de 10 minutos
        self.df_info = self.df_info.astype(
            {"status": str}
        )  # para evitar erros de comparação
        self.df_info.loc[
            (self.df_info["status"] == "true")
            & (self.df_info["tempo_registro_min"] <= 10),
            "status",
        ] = "in_test"

        # Ajustar nomenclatura dos status
        self.df_info.loc[
            self.df_info["status"] == "true", "status"
        ] = "rodando"
        self.df_info.loc[
            self.df_info["status"] == "false", "status"
        ] = "parada"

        # Remover colunas desnecessárias
        self.df_info.drop(
            columns=[
                "ciclo_1_min",
                "ciclo_15_min",
                "contagem_total_ciclos",
                "contagem_total_produzido",
                "tempo_parada",
                "tempo_rodando",
            ],
            inplace=True,
        )

        # Ajustar o index
        self.df_info.reset_index(drop=True, inplace=True)

        return self.df_info

    def clean_maq_occ(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de cadastro das máquinas

        Args:
        -----
            data (pd.DataFrame): Dataframe com os dados a serem limpos

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados limpos

        Exemplo:


        """

        self.df_occ = data.copy()

        # Motivos de Parada
        motivos = {
            1: "Ajustes",
            2: "Troca de Bobina",
            3: "Refeição",
            4: "Reunião",
            5: "Café e Ginástica Laboral",
            6: "Limpeza",
            7: "Manutenção Elétrica",
            8: "Manutenção Mecânica",
            9: "Material em Falta",
            10: "Setup de Sabor",
            11: "Setup de Tamanho",
            12: "Parada Programada",
            13: "Intervenção de Qualidade",
            14: "Linha Cheia",
            15: "Treinamento",
            16: "Limpeza Industrial",
        }

        # Unir as colunas de data e hora
        self.df_occ["data_hora_registro"] = (
            self.df_occ["data_registro"].astype(str)
            + " "
            + self.df_occ["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Modificar coluna motivo_id para int
        self.df_occ = self.df_occ.astype({"motivo_id": int})

        # Criar coluna motivo_nome com base no dicionário motivos
        self.df_occ["motivo_nome"] = self.df_occ["motivo_id"].map(motivos)

        # Ajustar problema se a string estiver vazia
        self.df_occ["problema"] = self.df_occ["problema"].replace("", np.nan)

        # Se o problema for nulo, copiar o motivo_nome para o problema, exceto para os motivos 1, 7, 8, 9, 14,
        self.df_occ.loc[
            self.df_occ["problema"].isnull()
            & ~self.df_occ["motivo_id"].isin([1, 7, 8, 9, 14]),
            "problema",
        ] = self.df_occ["motivo_nome"]

        # Definir ordem das colunas
        self.df_occ = self.df_occ[
            [
                "maquina_id",
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro",
                "usuario_id",
            ]
        ]

        return self.df_occ

Saída de Limpeza do Cadastro


In [48]:
clean_data = CleanData()

df_maq_cadastro_clean = clean_data.clean_maq_cadastro(df_maq_cadastro)

df_maq_cadastro_clean

,maquina_id,fabrica,linha,usuario_id,data_hora_registro
0,TMF001,1,8,002131,2023-09-25 10:45:38
1,TMF002,1,2,000264,2023-09-30 16:33:59
2,TMF003,1,5,000264,2023-10-06 15:54:22
3,TMF003,1,2,001996,2023-09-23 15:24:20
4,TMF004,1,6,001996,2023-09-25 11:52:29
5,TMF004,1,6,001996,2023-09-23 15:49:03
6,TMF005,1,1,000264,2023-11-11 12:43:59
7,TMF005,1,1,000264,2023-10-29 11:43:59
8,TMF005,1,9,000264,2023-10-14 14:57:44
9,TMF005,1,5,001996,2023-09-23 15:47:19


Saída da Limpeza de Máquina Info


In [49]:
df_maq_info_clean = clean_data.clean_maq_info(df_info)

df_maq_info_clean.head(25)

,maquina_id,status,turno,data_hora_registro,data_hora_final,tempo_registro_min
0,TMF001,parada,NOT,2024-01-01 00:02:21,2024-01-01 08:02:22,480
1,TMF001,parada,MAT,2024-01-01 08:02:22,2024-01-01 16:02:24,480
2,TMF001,parada,VES,2024-01-01 16:02:24,2024-01-02 00:02:25,480
3,TMF001,parada,NOT,2024-01-02 00:02:25,2024-01-02 08:02:27,480
4,TMF001,parada,MAT,2024-01-02 08:02:27,2024-01-02 16:02:28,480
5,TMF001,parada,VES,2024-01-02 16:02:28,2024-01-02 22:48:29,406
6,TMF001,in_test,VES,2024-01-02 22:48:29,2024-01-02 22:50:29,2
7,TMF001,parada,VES,2024-01-02 22:50:29,2024-01-02 23:00:29,10
8,TMF001,in_test,VES,2024-01-02 23:00:29,2024-01-02 23:02:29,2
9,TMF001,parada,VES,2024-01-02 23:02:29,2024-01-02 23:18:29,16


Saída da Limpeza das Ocorrências


In [50]:
df_maq_occ_clean = clean_data.clean_maq_occ(df_occ)

df_maq_occ_clean.head(25)

,maquina_id,motivo_id,motivo_nome,problema,solucao,data_hora_registro,usuario_id
0,TMF006,12,Parada Programada,Parada Programada,,2024-01-02 01:16:52,000453
1,TMF013,8,Manutenção Mecânica,NaN,,2024-01-02 01:17:04,000453
2,TMF010,8,Manutenção Mecânica,NaN,,2024-01-02 01:17:20,000453
3,TMF010,12,Parada Programada,Parada Programada,,2024-01-02 01:18:45,000453
4,TMF006,12,Parada Programada,Parada Programada,,2024-01-03 08:14:58,000807
5,TMF013,1,Ajustes,NaN,,2024-01-03 08:15:24,000807
6,TMF012,1,Ajustes,NaN,,2024-01-03 08:15:46,000807
7,TMF007,1,Ajustes,NaN,,2024-01-03 08:15:54,000807
8,TMF010,12,Parada Programada,Parada Programada,,2024-01-03 08:16:00,000807
9,TMF011,12,Parada Programada,Linha backup,,2024-01-03 08:29:32,000838


## Unindo info e ocorrências


join_data.py


In [57]:
class JoinData:
    """
    Classe para unir os dados de cadastro, info e ocorrência
    """

    def __init__(self):
        self.df_info = pd.DataFrame()
        self.df_cadastro = pd.DataFrame()
        self.df_occ = pd.DataFrame()
        self.df_join = pd.DataFrame()

    def join_info_occ(
        self, occ: pd.DataFrame, info: pd.DataFrame
    ) -> pd.DataFrame:
        """
        Une os dados de info e ocorrência
        """

        self.df_info = info.copy()
        self.df_occ = occ.copy()

        # Criar uma função para ser usada em cada linha do dataframe
        def merge_rows(row):
            # Selecionar rows onde a data_hora_registro de occ está entre
            # data_hora_registro e data_hora_final de info
            mask = (
                (
                    self.df_occ["data_hora_registro"]
                    >= row["data_hora_registro"]
                )
                & (self.df_occ["data_hora_registro"] <= row["data_hora_final"])
                & (self.df_occ["maquina_id"] == row["maquina_id"])
            )

            # Se houver rows selecionadas, retornar uma serie contendo os valores
            if self.df_occ.loc[mask].shape[0] > 0:
                return pd.Series(
                    [
                        self.df_occ.loc[mask, "motivo_id"].values[0],
                        self.df_occ.loc[mask, "motivo_nome"].values[0],
                        self.df_occ.loc[mask, "problema"].values[0],
                        self.df_occ.loc[mask, "solucao"].values[0],
                        self.df_occ.loc[mask, "data_hora_registro"].values[0],
                        self.df_occ.loc[mask, "usuario_id"].values[0],
                    ]
                )
            else:
                return pd.Series([None, None, None, None, None, None])

        # Aplicar a função merge_rows em cada linha do dataframe
        self.df_info[
            [
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro_operador",
                "usuario_id",
            ]
        ] = self.df_info.apply(merge_rows, axis=1)

        # Ajustar para sempre que estiver parada por motivo
        # 3, 4, 5, 12, 15, 16, o status será rodando
        self.df_info.loc[
            (self.df_info["status"] == "in_test")
            & (self.df_info["motivo_id"].shift(1).isin([3, 4, 5, 12, 15, 16])),
            "status",
        ] = "rodando"

        # Ajustar o index
        self.df_info.reset_index(drop=True, inplace=True)

        return self.df_info

Saída de join_info_occ


In [58]:
join_data = JoinData()

df_join = join_data.join_info_occ(df_maq_occ_clean, df_maq_info_clean)

df_join[25:50]

,maquina_id,status,turno,data_hora_registro,data_hora_final,tempo_registro_min,motivo_id,motivo_nome,problema,solucao,data_hora_registro_operador,usuario_id
25,TMF001,rodando,NOT,2024-01-03 02:40:30,2024-01-03 02:52:30,12,NaN,None,None,None,None,None
26,TMF001,parada,NOT,2024-01-03 02:52:30,2024-01-03 04:04:30,72,NaN,None,None,None,None,None
27,TMF001,rodando,NOT,2024-01-03 04:04:30,2024-01-03 08:02:31,238,NaN,None,None,None,None,None
28,TMF001,rodando,MAT,2024-01-03 08:02:31,2024-01-03 09:08:31,66,NaN,None,None,None,None,None
29,TMF001,parada,MAT,2024-01-03 09:08:31,2024-01-03 09:10:31,2,NaN,None,None,None,None,None
30,TMF001,rodando,MAT,2024-01-03 09:10:31,2024-01-03 09:54:31,44,NaN,None,None,None,None,None
31,TMF001,parada,MAT,2024-01-03 09:54:31,2024-01-03 10:00:31,6,8.0,Manutenção Mecânica,Falha pneumática,,2024-01-03 09:58:05,000838
32,TMF001,rodando,MAT,2024-01-03 10:00:31,2024-01-03 10:54:31,54,2.0,Troca de Bobina,Troca de produto,,2024-01-03 10:51:20,000838
33,TMF001,parada,MAT,2024-01-03 10:54:31,2024-01-03 11:00:31,6,NaN,None,None,None,None,None
34,TMF001,in_test,MAT,2024-01-03 11:00:31,2024-01-03 11:02:31,2,NaN,None,None,None,None,None
